In [204]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [205]:
load_dotenv()

True

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/api-key/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [207]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [208]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1752105602,
 'time_last_update_utc': 'Thu, 10 Jul 2025 00:00:02 +0000',
 'time_next_update_unix': 1752192002,
 'time_next_update_utc': 'Fri, 11 Jul 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.7264}

In [209]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.73})

857.3000000000001

In [210]:
# tool binding
llm = ChatOpenAI()

In [211]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [212]:
# Tool Calling
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 inr to usd')]

In [213]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [214]:
ai_message = llm_with_tools.invoke(messages)

In [215]:
messages.append(ai_message)

In [216]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WupVcDCBT1LZgPIMjwo81dOb', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_409HDyUkPrqhHbQy1ehlqyfz', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrfBIdtdF0aNHIHQtCxPpOOhOKzMm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--90732632-5871-4ea8-8b46-f0480e5eed23-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_cur

In [217]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_WupVcDCBT1LZgPIMjwo81dOb',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_409HDyUkPrqhHbQy1ehlqyfz',
  'type': 'tool_call'}]

In [218]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [219]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WupVcDCBT1LZgPIMjwo81dOb', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_409HDyUkPrqhHbQy1ehlqyfz', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrfBIdtdF0aNHIHQtCxPpOOhOKzMm', 'service_tier': 'defa

In [220]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 85.7264. \n\nConverting 10 INR to USD results in 857.264 USD.'

In [221]:
message

[HumanMessage(content='What is the converstion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1752105602, "time_last_update_utc": "Thu, 10 Jul 2025 00:00:02 +0000", "time_next_update_unix": 1752192002, "time_next_update_utc": "Fri, 11 Jul 2025 00:00:02 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 85.7264}', name='get_converstion_factor', tool_call_id='call_NtyitcJBVMy0XHYNf7WmIotK'),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1752105602, "time_last_update_utc": "Thu, 10 Jul 2025 00:00:02 +0000", "time_next_update_unix": 1752192002, "time_next_update_utc": "Fri, 11 Ju